In [1]:
import pandas as pd
import numpy as np
import itertools
import datetime
import matplotlib.pyplot as plt
from datetime import *
from datetime import datetime, timedelta, date
# %load_ext google.colab.data_table
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Standard plotly imports
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
# from google.colab import auth
# auth.authenticate_user()
# print('Authenticated')

In [4]:
#Change the date to the previous day
query = """
WITH Schedule as
(
SELECT
EXTRACT(DATE FROM Session5Date) Date, Year, RoundNumber, EventName, EventFormat
FROM `perceptive-ivy-290216.f1_api.schedules_uodate`
WHERE Session5='Race'
GROUP BY 1,2,3,4,5
ORDER BY 2,1
)
,

Results as (
SELECT
DriverNumber, Abbreviation, TeamName,FullName, CAST(Position as INT64) Position,CAST(GridPosition as INT64) GridPosition, Time, Status, CAST(Points as INT64) Points, A.Year, GP, B.Date, B.RoundNumber, B.EventFormat
FROM `perceptive-ivy-290216.f1_api.results_race` A
JOIN schedule B
ON A.GP=B.EventName AND A.Year=B.Year
# WHERE A.Year=2016
ORDER BY FullName, RoundNumber
)

SELECT
DriverNumber, Abbreviation, TeamName,FullName, Position, GridPosition, Time, Status, Points,
SUM(Points) OVER (PARTITION BY Year, FullName ORDER BY ROUNDNUMBER) PTS,
Year, GP, Date, RoundNumber, EventFormat
FROM
Results
ORDER BY FullName, RoundNumber

"""
project_id = "perceptive-ivy-290216"
df_bq = pd.read_gbq(query=query, project_id=project_id, dialect='standard')

/var/folders/x_/b65sxrpx6737wtqnt0ctv71w0000gn/T/ipykernel_5876/635947984.py:34: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_bq = pd.read_gbq(query=query, project_id=project_id, dialect='standard')


In [1]:
df_bq_driver=df_bq[df_bq["Year"]==2021]
df_bq_driver.head()

In [8]:
fig=px.line(df_bq_driver, x="Date", y="PTS", color='FullName', markers=True,
        hover_data=['DriverNumber','TeamName', 'Position', 'GridPosition', 'Status', 'Year', 'GP', 'RoundNumber'], height=1000, width=2000,

            color_discrete_map={
                 "HAM": "#6CD3BF",
                 "VER": "#3860C6",
                 "OCO": "#2293D1",
                 "NOR": "#F58020",
                 "MAG": "#B6BABD",
                 "ALB": "#37BEDD",
                 "BOT": "#C92D4B",
                 "LEC": "#F91536",
                 "VET": "#358C75",
                 "GAS": "#5E8FAA",
                 "RUS": "#6CD3BF",
                 "PER": "#3860C6",
                 "ALO": "#2293D1",
                 "RIC": "#F58020",
                 "MSC": "#B6BABD",
                 "LAT": "#37BEDD",
                 "ZHO": "#C92D4B",
                 "SAI": "#F91536",
                 "STR": "#358C75",
                 "TSU": "#5E8FAA"
                 }
        )
fig.update_layout(
    yaxis = dict(tickfont = dict(size=15)),
    xaxis = dict(tickfont = dict(size=15)),
    font=dict(
        family="PT Sans Narrow",
        size=14,
        color="Black"
    ),
    title_font_family="PT Sans Narrow"
#  "Arial", "Balto", "Courier New", "Droid Sans", "Droid Serif", "Droid Sans Mono", "Gravitas One", "Old Standard TT", "Open Sans", "Overpass", "PT Sans Narrow", "Raleway", "Times New Roman"
)

In [9]:
df_team=df_bq[df_bq["Year"]==2021]
df_team=df_team.groupby(by=["TeamName","Date","GP","Year","EventFormat","RoundNumber"])["Points","PTS"].sum()
df_team=pd.DataFrame(df_team)
df_team.reset_index(inplace=True)
df_team.head()

ValueError: Cannot subset columns with a tuple with more than one element. Use a list instead.

In [ ]:
px.line(df_team, x="Date", y="PTS", color='TeamName', markers=True,
        hover_data=['TeamName', 'Date', 'GP', 'Year', 'EventFormat', 'RoundNumber'], height=1000, width=2000,
        color_discrete_map={
                 "Mercedes": "#6CD3BF",
                 "Red Bull Racing": "#3671C6",
                 "Alpine": "#2293D1",
                 "McLaren": "#FFA500",
                 "Haas": "#B6BABD",
                 "Williams": "#37BEDD",
                 "Alfa Romeo Racing": "#C92D4B",
                 "Ferrari": "#F91536",
                 "Aston Martin": "#358C75",
                 "AlphaTauri": "#5E8FAA" }
        )